### Google Maps API Calling Script 1

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import json
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import scipy
import sys
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import emoji
import requests
import googlemaps
import warnings

In [2]:
# reading in api key and loading this in
f = open("api.txt", "r")

gmaps = googlemaps.Client(key=f.readlines()[1])
f.close()

In [4]:
cocotte_test = gmaps.find_place(input='Cocotte Notting Hill, London',
                                input_type='textquery',
                                fields=['name','place_id','formatted_address'],
                                location_bias='ipbias',
                                language='en')

In [5]:
brewery_test = gmaps.find_place(input='The Old Brewery, London',
                                input_type='textquery',
                                fields=['name','place_id','formatted_address'],
                                location_bias='ipbias',
                                language='en')

In [6]:
print(cocotte_test['candidates'])
print(brewery_test['candidates'])

[{'formatted_address': '95 Westbourne Grove, London W2 4UW, United Kingdom', 'name': 'Cocotte Notting Hill', 'place_id': 'ChIJJeZ74wEQdkgRRaSxXA9ggsM'}]
[{'formatted_address': 'The Pepys Building, The Old Royal Naval College, London SE10 9NN, United Kingdom', 'name': 'Old Brewery', 'place_id': 'ChIJGdIp0YQCdkgRvfDciFkSA_w'}]


In [8]:
print(cocotte_test['candidates'][0]['formatted_address'])

95 Westbourne Grove, London W2 4UW, United Kingdom


In [13]:
dict_list = [cocotte_test['candidates'][0], brewery_test['candidates'][0]]

df = pd.DataFrame()
for i in range(2):
    dict_selected = dict_list[i]
    df = df.append(dict_selected, ignore_index=True)
print(df)



                                   formatted_address                  name  \
0  95 Westbourne Grove, London W2 4UW, United Kin...  Cocotte Notting Hill   
1  The Pepys Building, The Old Royal Naval Colleg...           Old Brewery   

                      place_id  
0  ChIJJeZ74wEQdkgRRaSxXA9ggsM  
1  ChIJGdIp0YQCdkgRvfDciFkSA_w  


C:\Users\ndsna\AppData\Local\Temp\ipykernel_7392\3839055383.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict_selected, ignore_index=True)
C:\Users\ndsna\AppData\Local\Temp\ipykernel_7392\3839055383.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict_selected, ignore_index=True)


In [26]:
# import initial data to get a unique list of the restaurants in question which I can then call from
initial_df = pd.read_csv('Data/London_reviews.csv')
initial_df = initial_df.dropna(subset=['url_restaurant'])

C:\Users\ndsna\AppData\Local\Temp\ipykernel_12564\1209621210.py:2: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  initial_df = pd.read_csv('Data/London_reviews.csv')


In [27]:
# getting the unique restaurant names
restaurant_list = initial_df['restaurant_name'].unique()
# replacing the underscores between words with whitespaces. Also adding in ', London' to give more specificity over the location
restaurant_list = [(str(s).replace('_', ' ')) + ', London' for s in restaurant_list]
pd.Series(restaurant_list).to_csv('Restaurant_Name_List_TA.csv')

In [28]:
for i in restaurant_list[860:861]:
    print(i)

Artist Residence Clubhouse, London


In [29]:
missing_restaurants = []

missing_dict = {'formatted_address' : 'missing', 'name' :'missing', 'place_id' : 'missing'}

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    df = pd.DataFrame()
    for i in restaurant_list:
        restaurant_details = gmaps.find_place(input=i,
                                    input_type='textquery',
                                    fields=['name','place_id','formatted_address'],
                                    location_bias='ipbias',
                                    language='en')
        # moving past any case where it cannot be found
        if restaurant_details['candidates']:
            # taking the first restaurant, if it can be found
            dict_details = restaurant_details['candidates'][0]
            df = df.append(dict_details, ignore_index=True)
        else:
            missing_restaurants += i
            df = df.append(missing_dict, ignore_index=True)
    print(df)




                                      formatted_address  \
0     95 Westbourne Grove, London W2 4UW, United Kin...   
1     Union Canal, Sheldon Square, London W2 6EP, Un...   
2            212 High Rd, London N2 9AY, United Kingdom   
3     275 Old Brompton Rd, London SW5 9JA, United Ki...   
4     HMP Brixton, Jebb Ave, Brixton Hill, London SW...   
...                                                 ...   
1822  37 St Martin's Ln, London WC2N 4ER, United Kin...   
1823  23-24 Gloucester Rd, South Kensington, London ...   
1824      21 Piccadilly, London W1J 0BH, United Kingdom   
1825  20/21 Leicester Square, London WC2H 7LE, Unite...   
1826  The Pepys Building, The Old Royal Naval Colleg...   

                              name                     place_id  
0             Cocotte Notting Hill  ChIJJeZ74wEQdkgRRaSxXA9ggsM  
1                 London Shell Co.  ChIJL5dqGYUbdkgRQ7cwcCeg_zw  
2                            GENZO  ChIJ-1A3NFMbdkgRAzDMdHvhCFI  
3       The Gojk Restaurant

In [30]:
df.to_csv('Restaurant_Location_Details_2.0.csv', index = False)

In [33]:
df['TA_Names'] = restaurant_list
df['TA_Names'] = df['TA_Names'].apply(lambda x: re.sub(r', London$', '', x))
df.to_csv('Restaurant_Location_Details_2.0.csv', index = False)